# **Artificial Neural Networks and Deep Learning**

---

## **Lecture 4: Timeseries Classification**

<img src="https://drive.google.com/uc?export=view&id=1URwMSSlsgI716DsPSQVJXhll3vaPgL7v" width="500"/>

## 🌐 **Google Drive Connection**

In [ ]:
from google.colab import drive
drive.mount("/gdrive")
current_dir = "/gdrive/My\\ Drive/[2025-2026]\\ AN2DL/Lecture\\ 4"
%cd $current_dir

## ⚙️ **Libraries Import**

In [ ]:
# Set seed for reproducibility
SEED = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(SEED)
random.seed(SEED)

# Import PyTorch
import torch
torch.manual_seed(SEED)
from torch import nn
# from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
logs_dir = "tensorboard"
!pkill -f tensorboard
%load_ext tensorboard
!mkdir -p models

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

# Import other libraries
import copy
import shutil
from itertools import product
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

## ⏳ **Data Loading**

In [ ]:
# Set environment variables for Activity Recognition dataset
os.environ["DATASET_NAME"] = "activities_recognition.txt"
os.environ["DATASET_URL"] = "1QqyAJndGqPa-pWgyI63SuBAT__GnmL1b"

# Check if Activity Recognition dataset exists and download if not
if not os.path.exists(os.environ["DATASET_NAME"]):
    print("Downloading Activity Recognition dataset...")
    !gdown -q ${DATASET_URL} -O ${DATASET_NAME}
    print("Activity Recognition dataset downloaded!")
else:
    print("Activity Recognition dataset already downloaded. Using cached data.")

## 🔎 **Exploration and Data Analysis**

In [ ]:
# Define column names for the dataset
column_names = ['user_id', 'activity', 'timestamp', 'x_axis', 'y_axis', 'z_axis']

# Read the dataset into a DataFrame with specified column names
df = pd.read_csv('activities_recognition.txt', header=None, names=column_names)

# Remove rows with any missing values
df.dropna(axis=0, how='any', inplace=True)

# Print the shape of the DataFrame
print(f"DataFrame shape: {df.shape}")

# Display the first 10 rows of the DataFrame
df.head(10)

In [ ]:
# Display a concise summary of the DataFrame
df.info()

In [ ]:
# Convert 'x_axis' column to float32 data type
df['x_axis'] = df['x_axis'].astype(np.float32)

# Convert 'y_axis' column to float32 data type
df['y_axis'] = df['y_axis'].astype(np.float32)

# Convert 'z_axis' column to float32 data type
df['z_axis'] = df['z_axis'].astype(np.float32)

# Display updated DataFrame information
df.info()

In [ ]:
# Generate descriptive statistics for numerical columns in the DataFrame
df.describe()

In [ ]:
# Visualise the count of timestamps for each activity
plt.figure(figsize=(17, 5))
sns.countplot(
    x='activity',
    data=df,
    order=df['activity'].value_counts().index,
    palette='tab10'
)

# Set the title of the plot
plt.title('Activity Timestamps')

# Display the plot
plt.show()

In [ ]:
# Count the number of unique users
n_users = len(df['user_id'].unique())

# Create a custom colour map with distinct colours for each user
colors = plt.cm.viridis(np.linspace(0, 1, n_users))

# Visualise the count of timestamps for each user
plt.figure(figsize=(17, 5))
sns.countplot(
    x='user_id',
    data=df,
    palette=colors
)

# Set the title of the plot
plt.title('Per User Timestamps')

# Display the plot
plt.show()

In [ ]:
# Define a function to inspect sensor data for a specific activity
def inspect_activity(activity, df):
    # Filter the DataFrame for the specified activity and limit to 500 rows
    data = df[df['activity'] == activity][['x_axis', 'y_axis', 'z_axis']][:500]

    # Plot the sensor data for each axis
    axis = data.plot(subplots=True, figsize=(17, 9), title=activity)

    # Adjust legend position for each subplot
    for ax in axis:
        ax.legend(loc='lower right')

In [ ]:
# Inspect the sensor data for the activity "Standing"
inspect_activity("Standing", df)

In [ ]:
# Inspect the sensor data for the activity "Upstairs"
inspect_activity("Upstairs", df)

In [ ]:
# Inspect the sensor data for the activity "Jogging"
inspect_activity("Jogging", df)